<a href="https://colab.research.google.com/github/shriyanshsingh/A-Fine-Windy-Day/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [ ]:
# The following packages are installed by default on Colab
# If you're running on another platform, you may need to install them with
# pip, conda, brew, or other package managers.
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2024-12-06 01:32:43--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2024-12-06 01:32:43--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  10.4MB/s    in 0.4s    

2024-12-06 01:32:44 (10.4 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks we discussed in class.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8372730016708374),
 ('prince', 0.7308457493782043),
 ('duke', 0.7141456007957458),
 ('widow', 0.704542875289917),
 ('york', 0.6991268396377563),
 ('princess', 0.6934213042259216),
 ('warwick', 0.6918945908546448),
 ('cardinal', 0.6858773231506348),
 ('son', 0.6751499772071838),
 ('suffolk', 0.6736131310462952)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [ ]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.943556547164917),
 ('iago', 0.938313901424408),
 ('jul', 0.9118384718894958),
 ('imogen', 0.9086349606513977),
 ('ham', 0.9085931181907654),
 ('emilia', 0.9075225591659546),
 ('rosalind', 0.9025547504425049),
 ('valentine', 0.8985823392868042),
 ('glou', 0.892645537853241),
 ('countess', 0.8885270357131958)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

0.94355655

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
def cosim(v1, v2):
  dot_product = np.dot(v1, v2)
  norm_v1 = np.linalg.norm(v1)
  norm_v2 = np.linalg.norm(v2)
  return dot_product / (norm_v1 * norm_v2)

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9435564

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2024-12-06 01:33:01--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2024-12-06 01:33:02--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  39.1MB/s    in 1.7s    

2024-12-06 01:33:04 (39.1 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2024-12-06 01:33:04--  http://www.ccs.neu.edu/home/dasmith/courses/

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [ ]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [ ]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  mostSimilar = ''
  similarity = -1
  for word, vector in vecDict.items():
      current_similarity = cosim(vec, vector)
      if current_similarity > similarity:
        similarity = current_similarity
        mostSimilar = word
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2024-12-06 01:36:34--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/links.tab [following]
--2024-12-06 01:36:35--  https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.khoury.northeastern.edu (www.khoury.northeastern.edu)... 52.70.229.197
Connecting to www.khoury.northeastern.edu (www.khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  3.59MB/s    in 0.4s    

2024-12-06 01:36:36 (3.59 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French from the first 1000 links between those languages.

In [ ]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
correct = 0
baseline_correct = 0
total = 0

for en_word, _, fr_word in frtest:
    if en_word in envec and fr_word in frvec:
        total += 1
        predicted_word, _ = mostSimilar(envec[en_word], frvec)
        if predicted_word == fr_word:
            correct += 1
        if en_word == fr_word:
            baseline_correct += 1

accuracy = correct / total if total > 0 else 0
baselineAccuracy = baseline_correct / total if total > 0 else 0

print(f"Embedding accuracy: {accuracy:.4f}")
print(f"Baseline accuracy: {baselineAccuracy:.4f}")

Embedding accuracy: 0.8443
Baseline accuracy: 0.6065


**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
estest = [x for x in links if x[1] == "es"][0:1000]

correct_es = 0
baseline_correct_es = 0
total_es = 0

for en_word, _, es_word in estest:
    if en_word in envec and es_word in esvec:
        total_es += 1
        predicted_word, _ = mostSimilar(envec[en_word], esvec)
        if predicted_word == es_word:
            correct_es += 1
        if en_word == es_word:
            baseline_correct_es += 1

accuracy_es = correct_es / total_es if total_es > 0 else 0
baselineAccuracy_es = baseline_correct_es / total_es if total_es > 0 else 0

print(f"Embedding accuracy (ES): {accuracy_es:.4f}")
print(f"Baseline accuracy (ES): {baselineAccuracy_es:.4f}")

Embedding accuracy (ES): 0.8284
Baseline accuracy (ES): 0.4590


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [ ]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.
def compute_metrics(test_set, src_vec, tgt_vec):
    recall_10 = 0
    mrr_10 = 0
    total = 0

    for src_word, _, tgt_word in test_set:
        if src_word in src_vec and tgt_word in tgt_vec:
            total += 1
            neighbors = sorted(tgt_vec.items(), key=lambda x: cosim(src_vec[src_word], x[1]), reverse=True)[:10]
            neighbor_words = [word for word, _ in neighbors]

            if tgt_word in neighbor_words:
                recall_10 += 1
                rank = neighbor_words.index(tgt_word) + 1
                mrr_10 += 1 / rank

    recall_10 /= total if total > 0 else 1
    mrr_10 /= total if total > 0 else 1

    return recall_10, mrr_10

# Compute metrics for French
recall_10_fr, mrr_10_fr = compute_metrics(frtest, envec, frvec)
print(f"French - Recall@10: {recall_10_fr:.4f}, MRR@10: {mrr_10_fr:.4f}")

# Compute metrics for Spanish (or your chosen language)
recall_10_es, mrr_10_es = compute_metrics(estest, envec, esvec)
print(f"Spanish - Recall@10: {recall_10_es:.4f}, MRR@10: {mrr_10_es:.4f}")

French - Recall@10: 0.9457, MRR@10: 0.8831
Spanish - Recall@10: 0.9490, MRR@10: 0.8728


## Speeding up Vector Search (for CS6200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [ ]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss-gpu
import time
import numpy as np
import faiss

def create_faiss_index(vectors):
    d = vectors[list(vectors.keys())[0]].shape[0]  # Get the dimension of vectors
    index = faiss.IndexFlatIP(d)  # Use inner product, which is equivalent to cosine similarity for normalized vectors

    # Normalize vectors and add to the index
    normalized_vectors = []
    for vec in vectors.values():
        normalized_vec = vec / np.linalg.norm(vec)
        normalized_vectors.append(normalized_vec)

    index.add(np.array(normalized_vectors))
    return index, list(vectors.keys())

# Create Faiss indexes
fr_index, fr_words = create_faiss_index(frvec)
es_index, es_words = create_faiss_index(esvec)

def faiss_search(query_vec, index, words, k=10):
    normalized_query = query_vec / np.linalg.norm(query_vec)
    D, I = index.search(normalized_query.reshape(1, -1), k)
    return [words[i] for i in I[0]]

# Measure effectiveness and efficiency
def evaluate_search(test_set, src_vec, tgt_index, tgt_words):
    recall_10 = 0
    mrr_10 = 0
    total = 0
    start_time = time.time()

    for src_word, _, tgt_word in test_set:
        if src_word in src_vec:
            total += 1
            neighbors = faiss_search(src_vec[src_word], tgt_index, tgt_words)

            if tgt_word in neighbors:
                recall_10 += 1
                rank = neighbors.index(tgt_word) + 1
                mrr_10 += 1 / rank

    elapsed_time = time.time() - start_time
    recall_10 /= total if total > 0 else 1
    mrr_10 /= total if total > 0 else 1

    return recall_10, mrr_10, elapsed_time

# Evaluate French
recall_10_fr_faiss, mrr_10_fr_faiss, time_fr_faiss = evaluate_search(frtest, envec, fr_index, fr_words)
print(f"French (Faiss) - Recall@10: {recall_10_fr_faiss:.4f}, MRR@10: {mrr_10_fr_faiss:.4f}, Time: {time_fr_faiss:.2f}s")

# Evaluate Spanish
recall_10_es_faiss, mrr_10_es_faiss, time_es_faiss = evaluate_search(estest, envec, es_index, es_words)
print(f"Spanish (Faiss) - Recall@10: {recall_10_es_faiss:.4f}, MRR@10: {mrr_10_es_faiss:.4f}, Time: {time_es_faiss:.2f}s")

# Compare with brute-force search times
start_time = time.time()
recall_10_fr, mrr_10_fr = compute_metrics(frtest, envec, frvec)
time_fr_brute = time.time() - start_time
print(f"French (Brute-force) - Time: {time_fr_brute:.2f}s")

start_time = time.time()
recall_10_es, mrr_10_es = compute_metrics(estest, envec, esvec)
time_es_brute = time.time() - start_time
print(f"Spanish (Brute-force) - Time: {time_es_brute:.2f}s")

print(f"Speedup for French: {time_fr_brute / time_fr_faiss:.2f}x")
print(f"Speedup for Spanish: {time_es_brute / time_es_faiss:.2f}x")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.7 MB/s eta 0:00:00
French (Faiss) - Recall@10: 0.6440, MRR@10: 0.6014, Time: 4.07s
Spanish (Faiss) - Recall@10: 0.6140, MRR@10: 0.5647, Time: 3.61s
French (Brute-force) - Time: 215.68s
Spanish (Brute-force) - Time: 205.76s
Speedup for French: 53.05x
Speedup for Spanish: 57.00x


**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)